## Pair Programming APIs

1. Extracción de Datos a Nivel Nacional:

- Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual.

- Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.

- Almacenar los datos en un DataFrame con la siguiente estructura.

In [1]:
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)

In [2]:
def llamar_api(url):
    
    llamada = requests.get(url)
    
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")
    
    if llamada.status_code != 200:
        print(f"El motivo por el que la llamada falló es {llamada.reason} {llamada.text}")
    else:
        return llamada.json()  

In [3]:
url_energia = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date=2021-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month"

resultados_energia = llamar_api(url_energia)

La llamada a la API nos ha dado una respuesta de tipo: 200


In [24]:
diccionario_energia = {"tipo_energia": [], "tipo_generacion": [], "fecha": [], "valor": []}

for año in range(2021, 2022): 
    url_nacional = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month"
    datos_año =  requests.get(url_nacional)
    json_año = datos_año.json()

    for elemento in json_año["included"]:
        for dato in elemento["attributes"]["values"]:
            diccionario_energia["tipo_energia"].append(elemento["attributes"]["title"])
            diccionario_energia["tipo_generacion"].append(elemento["attributes"]["type"])
            diccionario_energia["valor"].append(dato["value"])
            diccionario_energia["fecha"].append(dato["datetime"])

df_energia = pd.DataFrame(diccionario_energia)

df_energia          

,tipo_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2021-01-01T00:00:00.000+01:00,4.057347e+06
1,Hidráulica,Renovable,2021-02-01T00:00:00.000+01:00,4.517602e+06
2,Hidráulica,Renovable,2021-03-01T00:00:00.000+01:00,3.715416e+06
3,Hidráulica,Renovable,2021-04-01T00:00:00.000+02:00,2.745019e+06
4,Hidráulica,Renovable,2021-05-01T00:00:00.000+02:00,2.159483e+06
...,...,...,...,...
202,Generación total,Generación total,2021-08-01T00:00:00.000+02:00,2.158524e+07
203,Generación total,Generación total,2021-09-01T00:00:00.000+02:00,2.068887e+07
204,Generación total,Generación total,2021-10-01T00:00:00.000+02:00,2.063464e+07
205,Generación total,Generación total,2021-11-01T00:00:00.000+01:00,2.301770e+07


2. Extracción de Datos por Comunidad Autónoma:

- Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.

- Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.

- Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.

- Almacenar los datos en un DataFrame con la siguiente estructura.

In [21]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [35]:
diccionario_ccaa = {"tipo_energia": [], "tipo_generacion": [], "fecha": [], "valor": [], "ccaa": []}

for k, v in cod_comunidades.items():
    
    for año in range(2021, 2022): 
        
        url_ccaa = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={v}"
        datos_ccaa =  requests.get(url_ccaa)
        json_ccaa = datos_ccaa.json()

        for elemento in json_ccaa["included"]:
            
            for dato in elemento["attributes"]["values"]:
                diccionario_ccaa["tipo_energia"].append(elemento["attributes"]["title"])
                diccionario_ccaa["tipo_generacion"].append(elemento["attributes"]["type"])
                diccionario_ccaa["valor"].append(dato["value"])
                diccionario_ccaa["fecha"].append(dato["datetime"])
                diccionario_ccaa["ccaa"].append(k)

In [36]:
df_ccaa = pd.DataFrame(diccionario_ccaa)

df_ccaa

,tipo_energia,tipo_generacion,fecha,valor,ccaa
0,Motores diésel,No-Renovable,2021-01-01T00:00:00.000+01:00,18537.330,Ceuta
1,Motores diésel,No-Renovable,2021-02-01T00:00:00.000+01:00,15212.184,Ceuta
2,Motores diésel,No-Renovable,2021-03-01T00:00:00.000+01:00,15333.781,Ceuta
3,Motores diésel,No-Renovable,2021-04-01T00:00:00.000+02:00,15113.023,Ceuta
4,Motores diésel,No-Renovable,2021-05-01T00:00:00.000+02:00,15508.211,Ceuta
...,...,...,...,...,...
1995,Generación total,Generación total,2021-08-01T00:00:00.000+02:00,123555.476,La Rioja
1996,Generación total,Generación total,2021-09-01T00:00:00.000+02:00,121050.798,La Rioja
1997,Generación total,Generación total,2021-10-01T00:00:00.000+02:00,136012.046,La Rioja
1998,Generación total,Generación total,2021-11-01T00:00:00.000+01:00,291093.344,La Rioja
